**Защита данные клиентов страховой компании**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.  
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Ход исследования<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Изучение-данных" data-toc-modified-id="1.-Изучение-данных-1">1. Изучение данных</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.1-Изучение-общей-информации" data-toc-modified-id="1.1-Изучение-общей-информации-1.0.1">1.1 Изучение общей информации</a></span></li></ul></li><li><span><a href="#1.2-Изучение-корреляции-данных" data-toc-modified-id="1.2-Изучение-корреляции-данных-1.1">1.2 Изучение корреляции данных</a></span></li><li><span><a href="#1.3-Исследование-баланса-классов" data-toc-modified-id="1.3-Исследование-баланса-классов-1.2">1.3 Исследование баланса классов</a></span></li></ul></li><li><span><a href="#2.-Подготовка-данных" data-toc-modified-id="2.-Подготовка-данных-2">2. Подготовка данных</a></span></li><li><span><a href="#3-Исследование-влияние-кодирования-данных-на-качество-модели" data-toc-modified-id="3-Исследование-влияние-кодирования-данных-на-качество-модели-3">3 Исследование влияние кодирования данных на качество модели</a></span><ul class="toc-item"><li><span><a href="#3.1-Обучение-линейной-регрессии-без-кодирования" data-toc-modified-id="3.1-Обучение-линейной-регрессии-без-кодирования-3.1">3.1 Обучение линейной регрессии без кодирования</a></span></li><li><span><a href="#3.2-Признаки-умножим-на-обратимую-матрицу" data-toc-modified-id="3.2-Признаки-умножим-на-обратимую-матрицу-3.2">3.2 Признаки умножим на обратимую матрицу</a></span></li><li><span><a href="#3.3-Обучение-линейной-регрессии-с-кодированием" data-toc-modified-id="3.3-Обучение-линейной-регрессии-с-кодированием-3.3">3.3 Обучение линейной регрессии с кодированием</a></span></li><li><span><a href="#3.4-Исследование-связи-параметров-линейной-регрессии-в-исходной-задаче-и-в-преобразованной" data-toc-modified-id="3.4-Исследование-связи-параметров-линейной-регрессии-в-исходной-задаче-и-в-преобразованной-3.4">3.4 Исследование связи параметров линейной регрессии в исходной задаче и в преобразованной</a></span></li><li><span><a href="#3.5-Как-раскодировать-данные" data-toc-modified-id="3.5-Как-раскодировать-данные-3.5">3.5 Как раскодировать данные</a></span></li><li><span><a href="#3.6-Описание-алгоритма-кодировки" data-toc-modified-id="3.6-Описание-алгоритма-кодировки-3.6">3.6 Описание алгоритма кодировки</a></span></li><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-3.7">Вывод.</a></span></li></ul></li></ul></div>

# 1. Изучение данных

### 1.1 Изучение общей информации

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
#from sklearn.preprocessing import OrdinalEncoder
#from sklearn.preprocessing import MinMaxScaler

In [2]:
try:
    data = pd.read_csv('D:\\Хранилище информации\\DATA SCIENCE учеба\\projects\\insurance.csv', sep=',', encoding='utf-8')
    print('Прочитано с локального диска')
except:
    data = pd.read_csv('https://code.s3.yandex.net//datasets//insurance.csv', sep=',')
display(data.head())

Прочитано с локального диска


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


•	Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.  
•	Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
# Проверим на дубликаты
data.duplicated().sum()

153

In [5]:
# посмотрим на распределение данных
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
# переименуем колонки
data = data.rename(columns = {'Пол':'sex', 'Возраст':'age', 'Зарплата':'salary', 'Члены семьи':'family_memebers',
                              'Страховые выплаты':'insuarance_payoff'})
data.head()

,sex,age,salary,family_memebers,insuarance_payoff
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
# Преобразуем в целочисленный формат
for column in data.columns:
    data[column] = data[column].astype('int32')
data.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   sex                5000 non-null   int32
 1   age                5000 non-null   int32
 2   salary             5000 non-null   int32
 3   family_memebers    5000 non-null   int32
 4   insuarance_payoff  5000 non-null   int32
dtypes: int32(5)
memory usage: 97.8 KB


Имеем: 5000 строк записей, с 5 колонками. Пропусков нет, имеются дубликаты, но т.к. это скорее вспего совпадения, удалять их не будем.

## 1.2 Изучение корреляции данных

In [8]:
data.corr()

,sex,age,salary,family_memebers,insuarance_payoff
sex,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
salary,0.014910,-0.019093,1.000000,-0.030296,-0.014963
family_memebers,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
insuarance_payoff,0.010140,0.651030,-0.014963,-0.036290,1.000000


Самая большая корреляция обнаружена у параметров возраста и страховых выплат - 65%.

## 1.3 Исследование баланса классов

Выведем количество и процентное значение целевого признака

In [9]:
series = data['insuarance_payoff'].value_counts()

In [10]:
df = pd.DataFrame([series])
df = df.T
df['share'] = df['insuarance_payoff'] / len(data) * 100
df

,insuarance_payoff,share
0,4436,88.72
1,423,8.46
2,115,2.30
3,18,0.36
4,7,0.14
5,1,0.02


Как видно, 88% наблюдений целевого признака являются 0.

# 2. Подготовка данных

Разделим данные на выборки в отношении 3 к 1.

In [11]:
target = data['insuarance_payoff']
features = data.drop('insuarance_payoff', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
print(f'Размер обучающей выборки: {features_train.shape}')
print(f'Размер валидационной выборки: {features_valid.shape}')

Размер обучающей выборки: (3750, 4)
Размер валидационной выборки: (1250, 4)


In [12]:
print(features_train.head())

      sex  age  salary  family_memebers
3369    1   43   36200                1
1441    1   34   57600                0
571     0   32   41100                1
225     0   36   45100                1
2558    0   33   50600                2


# 3 Исследование влияние кодирования данных на качество модели

## 3.1 Обучение линейной регрессии без кодирования

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print("R2 =", r2_score(target_valid,predicted_valid))

R2 = 0.43522756840832744


## 3.2 Признаки умножим на обратимую матрицу

Обратимая матрица - это матрица, у которой есть обратная. Результат умножения обратной матрицы на обратимую - единичная матрица.  Умножение матрицы на матрицу возможно, если ширина первой матрицы  равна высоте второй матрицы. Наша первая матрица - это признаки размером 5000 на 4, в результате хотим получить закодированные признаки - это матрицу размером тоже 5000 на 4, значит умножать будем на матрицу 4 на 4.  
Подберём такую обратимую матрицу.

In [14]:
P = np.array([[5, 9, 13, 6],
              [1, 47, 3, 19],
             [4, 7, 8, 2],
             [8, 4, 11, 3],])
b =np.linalg.inv(P) # вычислим обратную матрицу
b

array([[-0.19726859,  0.02579666, -0.08345979,  0.28679818],
       [-0.07934099,  0.01257316,  0.18621288, -0.04508996],
       [ 0.12117928, -0.03013223,  0.12269673, -0.13331888],
       [ 0.18751355,  0.02492955, -0.4756124 ,  0.11749404]])

На обратимость можно проверить еще вот так: Если определитель квадратной матрицы P не равен нулю, значит у нее есть обратная.

In [15]:
P.dot(b) # проверим точность обратной матрицы

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.11022302e-16],
       [-2.77555756e-16,  1.00000000e+00,  1.11022302e-16,
        -2.77555756e-17],
       [-1.11022302e-16,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [-1.66533454e-16,  2.77555756e-17,  0.00000000e+00,
         1.00000000e+00]])

Умножим признаки на обратимую матрицу.

In [16]:
features_coded = np.array(features).dot(P)
features_coded

array([[198454, 349140, 396947,  99988],
       [152054, 268166, 304149,  76877],
       [ 84029, 148363, 168087,  42551],
       ...,
       [135636, 238248, 271282,  68186],
       [130851, 229955, 261712,  65833],
       [162441, 285529, 324908,  81741]])

Для генерации полностью случайной матрицы размера 4*4 можно воспльзоваться следующим генератором:

In [17]:
np.random.normal(size=(4, 4))

array([[-1.3114606 , -0.27556422,  0.5636795 , -0.42583326],
       [ 0.64890949, -1.23795684, -0.1296062 , -0.27379265],
       [-0.12773987,  1.60050697, -1.69752333, -1.83156818],
       [-0.92701498,  0.04551115, -0.21600097,  0.32146723]])

In [18]:
np.random.seed(0)        # random.seed(0) позволит генерировать одну и ту же случайную матрицу
m = np.random.rand(4,4)   
m

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
       [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
       [0.96366276, 0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606, 0.0871293 ]])

## 3.3 Обучение линейной регрессии с кодированием

In [20]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_coded, target, test_size=0.25, random_state=12345)
print(f'Размер обучающей выборки: {features_train.shape}')
print(f'Размер валидационной выборки: {features_valid.shape}')

Размер обучающей выборки: (3750, 4)
Размер валидационной выборки: (1250, 4)


In [21]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print("R2 =", r2_score(target_valid,predicted_valid))

R2 = 0.43522756840832666


Как видно, качество линейной регрессии не изменилось.  
Разебём, почему так получилось. 

## 3.4 Исследование связи параметров линейной регрессии в исходной задаче и в преобразованной

Задача алгоритма обучения линейной регрессии сводится к уравнению: $a=Xw$, где $а$ - целевые признаки, $Х$ - обучающие признаки в виде матрицы, $w$ - искомый вектор предсказаний. Нужно найти такую $w$, чтобы функция потерь была минимальна.  
Веса $w$ вычисляются по формуле: $w = (Х^Т  Х)^{-1}   Х^Т  y$

При умножении признаков на матрицу $Р$ получаем новые веса $w_1$:  
$w_1 =  ( (ХP)^Т  (ХP))^{-1}  (ХP)^Т  y$  

Для того, чтобы умножение матриц было возможно, из размеры должны быть согласованны. Обозначим размеры матриц в уравнении как:  
$m$ - количество строк матрицы признаков (в наших данных $m$=5000)  
$n$ - количество столбцов матрицы признаков (в наших данных $n$=4)  
$k$ - количество столбцов матрицы целевого признака (в наших данных $k$=1)

$w_1 =  ( (ХP)^Т  (ХP))^{-1}  (ХP)^Т  y$     
Размеры матриц:     
$Х$ - $mxn$  \
$Р$ - $nxn$  \
$y$ - $mxk$  \
$X^T$ - $nxm$ - транспонированная матрица от $X$

$((ХP)^Т (ХP))^{-1} (ХP)^Т  y $ =   

Воспользуемся свойством матриц: $(АВ)^T = В^T А^T$ :     
 
 = $(P^Т Х^Т ХP )^{-1} (ХP)^Т  y$ 

 
Воспользуемся свойством матриц: $(АВC)^{-1} = C^{-1} В^{-1}  А^{-1}$ , следя за тем, чтобы все матрицы, для которых ищутся обратые - операция $^{-1}$, были квадратными:

 = $ P ^{-1}  (Х^Т Х)^{-1} (P^Т)^{-1} (ХP)^Т  y$ = 
 
Такая операция возможна, т.к. $P$, $P^Т$ - квадратные матрицы по условию задачи, и $(Х^Т Х)$ - тоже квадратная матрица. \
 
Воспользуемся свойством матриц: $(АВ)^T = В^T А^T$:   
 
 =  $ P ^{-1}  (Х^Т Х)^{-1} (P^Т)^{-1} P^Т X^Т y$ =   
 
 
Укажем размерность матриц этой строки:

$(nxn)  (nxm * mxn)^{-1}   (nxn)^{-1}    (nxn)   (nxm)  (mxk)$   

Как видно, все матрицы в выражении согласованы.

В уравнении выражение $(P^Т)^{-1} P^T  = Е$ , что даёт при умножении любой матрицы её саму , при условии согласования размера, размер получившейся матрицы $Е$ будет $n*n$, что позволяет умножить ее на стоящий справа член:

=  $ P ^{-1}  (Х^Т Х)^{-1} E X^Т y$ = 

=  $ P ^{-1}  (Х^Т Х)^{-1}  X^Т y$ = 


=  $ P ^{-1}  w$  .

Получаем:  
$w_1 = P ^{-1}  w $

Для уравнения регрессии с закодированными признаками получаем:  <br>
$a = X  Р  w_1$  
Подставим выведенное выражение $w_1 = P ^{-1}  w $:  
$a = X  Р  P ^{-1} w$  
$a = X  Е  w$  
$a = X   w$

Получено доказательство того, что умножение признаков на матрицу не влияет на качество линейной регресси.

## 3.5 Как раскодировать данные

Чтобы извлечь исходные данные из закодировнных, понадобиться решить матричное уравнение такого вида: X*a = c, где X - это признаки, подвергнутые кодировке, а - это наша обратимая матрица а, с - это закодированные данные.  
Матричное уравнение X*a = c решается так:
X*a * а(-1) = c * а(-1), где  а(-1) - обратная матрица к матрице а. Т.к. a * а(-1) = Е, а Х*Е = Х, получим:
X = c * а(-1)

In [22]:
features_coded.dot(b)

array([[ 1.00000000e+00,  4.10000000e+01,  4.96000000e+04,
         1.00000000e+00],
       [-1.81898940e-12,  4.60000000e+01,  3.80000000e+04,
         1.00000000e+00],
       [-1.36424205e-12,  2.90000000e+01,  2.10000000e+04,
         9.09494702e-13],
       ...,
       [-4.54747351e-12,  2.00000000e+01,  3.39000000e+04,
         2.00000000e+00],
       [ 1.00000000e+00,  2.20000000e+01,  3.27000000e+04,
         3.00000000e+00],
       [ 1.00000000e+00,  2.80000000e+01,  4.06000000e+04,
         1.00000000e+00]])

Мы можем убедиться, что полученные данные совпадают с исходными.

## 3.6 Описание алгоритма кодировки

 1. Генерируем матрицу размером n*n, где n - это количество признаков для обучения модели
 2. Умножаем матрицу на признаки
 3. Разделяем данные на обучающую и валидационную выборки.
 4. Обучаем модель

## Вывод. 
В данном проекте мы проверили на данных и обосновали теоретически утверждение о том, что при умножении признаков на матрицу качество алгоритма обучения не изменится. 